In [1]:
!pip install kaggle plotly hdbscan --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 23.1 MB/s eta 0:00:00


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

!kaggle datasets download -d faisalsanto007/isear-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/faisalsanto007/isear-dataset
License(s): ODbL-1.0
  0% 0.00/315k [00:00<?, ?B/s]
100% 315k/315k [00:00<00:00, 28.4MB/s]


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import hdbscan
from sklearn.decomposition import PCA
import plotly.express as px
import re

In [9]:
df=pd.read_csv("/content/eng_dataset.csv")

In [10]:
df.head()

,ID,sentiment,content
0,10941,anger,At the point today where if someone says somet...
1,10942,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...
2,10943,anger,This game has pissed me off more than any othe...
3,10944,anger,@spamvicious I've just found out it's Candice ...
4,10945,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...


In [11]:
df['content'] = (
    df['content']
    .astype(str)                        # Ensure it's string
    .str.replace(r'@\S+', '', regex=True)  # Remove @ + any non-whitespace chars
    .str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with single space
    .str.strip()                           # Strip leading/trailing whitespace
)

In [12]:
df.head()

,ID,sentiment,content
0,10941,anger,At the point today where if someone says somet...
1,10942,anger,IT'S GAME DAY!!!! T MINUS 14:30 #relentless
2,10943,anger,This game has pissed me off more than any othe...
3,10944,anger,I've just found out it's Candice and not Canda...
4,10945,anger,if he can't come to my Mum'a 60th after 25k tw...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7102 entries, 0 to 7101
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         7102 non-null   int64 
 1   sentiment  7102 non-null   object
 2   content    7102 non-null   object
dtypes: int64(1), object(2)
memory usage: 166.6+ KB


In [14]:
df['sentiment'].unique()

array(['anger', 'fear', 'joy', 'sadness'], dtype=object)

In [15]:
df.drop(columns=['ID'], errors='ignore',inplace=True)

In [16]:
baseline_df = df[df['sentiment'] == 'joy']

In [19]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

all_texts = df['content'].astype(str).tolist()
embeddings = model.encode(all_texts, convert_to_tensor=False)

baseline_texts = baseline_df['content'].astype(str).tolist()
baseline_embeddings = model.encode(baseline_texts, convert_to_tensor=False)

baseline_centroid = np.mean(baseline_embeddings, axis=0)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['cluster'] = labels

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [44]:
def compute_stress_score(embedding, baseline_centroid, normalization_factor=2.0):
    """
    Computes a stress score by calculating the Euclidean distance
    between the text's embedding and the baseline (joy) centroid.
    The final stress score is scaled to a range of [0, 5].
    """
    distance = np.linalg.norm(embedding - baseline_centroid)
    # Normalize the distance to [0, 1]
    score_0_1 = distance / normalization_factor
    score_0_1 = min(max(score_0_1, 0), 1)
    # Scale to [0, 5]
    stress_score_0_5 = 5 * score_0_1
    return stress_score_0_5

# Compute stress scores for all texts
stress_scores = [compute_stress_score(embed, baseline_centroid) for embed in embeddings]


df['stress_score'] = stress_scores

In [45]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(embeddings)
df_plot = pd.DataFrame(X_pca, columns=['X', 'Y', 'Z'])
df_plot['cluster'] = labels.astype(str)  # Convert labels to string for discrete coloring

# Create an interactive 3D scatter plot with Plotly
fig = px.scatter_3d(
    df_plot,
    x='X',
    y='Y',
    z='Z',
    color='cluster',
    title='HDBSCAN Clusters (PCA 3D)',
    width=800,
    height=600
)
fig.show()

In [46]:
df.head()

,sentiment,content,cluster,stress_score
0,anger,At the point today where if someone says somet...,-1,2.248889
1,anger,IT'S GAME DAY!!!! T MINUS 14:30 #relentless,213,2.282248
2,anger,This game has pissed me off more than any othe...,-1,2.342334
3,anger,I've just found out it's Candice and not Canda...,266,2.419071
4,anger,if he can't come to my Mum'a 60th after 25k tw...,-1,2.352784


In [47]:
df.drop(columns=['sentiment','cluster'],inplace=True)

In [48]:
df.head()

,content,stress_score
0,At the point today where if someone says somet...,2.248889
1,IT'S GAME DAY!!!! T MINUS 14:30 #relentless,2.282248
2,This game has pissed me off more than any othe...,2.342334
3,I've just found out it's Candice and not Canda...,2.419071
4,if he can't come to my Mum'a 60th after 25k tw...,2.352784


In [53]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

x=df['content']
y=df['stress_score']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [54]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_texts(texts):
    return model.encode(texts, convert_to_tensor=False)

# Convert training texts to embeddings
train_embeddings = embed_texts(x_train.tolist())
test_embeddings  = embed_texts(x_test.tolist())

In [55]:
regressor=RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(train_embeddings, y_train)

# Evaluate
y_pred = regressor.predict(test_embeddings)
print("MSE : ", mean_squared_error(y_test, y_pred))
print('MAE : ',mean_absolute_error(y_test,y_pred))
print("R2 score : ", r2_score(y_test,y_pred))

MSE :  0.003930079093350026
MAE :  0.04851680455793594
R2 score :  0.732568185770712


In [59]:
import joblib

joblib.dump(regressor, "stress_regressor.pkl")

['stress_regressor.pkl']